# Setup

In [1]:
import pandas as pd
import os
import datacompy
import numpy as np

#### This notebook creates the census 2024 file based off the 2023 file.  
##### The census 2023 file has the following updates applied to it.
* if median age = 2002, then set it to 6

# Import Data

In [14]:
# this census file includes includes both old and new CT county code rows
df_census_2023 = pd.read_csv("../output_FHFA/ffiec_census_2023_includes_new_CT_county_codes.txt",delimiter="|", dtype= {"State": str,   "County": str})

df_census_2024 = df_census_2023.copy()

# df_CT_map = pd.read_csv("../output_FHFA/connecticut_county_code_mapping.csv")

# Print first 5 rows

In [15]:
df_census_2024.columns

Index(['Collection Year', 'MSA/MD', 'State', 'County', 'Census Tract',
       'FFIEC Median Family Income', 'Population', 'Minority Population %',
       'Number of Owner Occupied Units', 'Number of 1 to 4 Family Units',
       'Tract MFI', 'Tract to MSA Income %', 'Median Age', 'Small County',
       'MSA/MD Name'],
      dtype='object')

In [16]:
df_census_2024.shape

(88182, 15)

In [4]:
df_census_2024.head()

,Collection Year,MSA/MD,State,County,Census Tract,FFIEC Median Family Income,Population,Minority Population %,Number of Owner Occupied Units,Number of 1 to 4 Family Units,Tract MFI,Tract to MSA Income %,Median Age,Small County,MSA/MD Name
0,2023,33860,01,001,20100,68115,1775.0,22.48,507.0,710.0,70699.0,103.79,40.0,T,"Montgomery, AL"
1,2023,33860,01,001,20200,68115,2055.0,59.42,392.0,717.0,50133.0,73.60,48.0,T,"Montgomery, AL"
2,2023,33860,01,001,20300,68115,3216.0,30.97,967.0,1401.0,70111.0,102.93,44.0,T,"Montgomery, AL"
3,2023,33860,01,001,20400,68115,4246.0,17.05,1290.0,1598.0,75580.0,110.95,48.0,T,"Montgomery, AL"
4,2023,33860,01,001,20501,68115,4322.0,25.71,1024.0,1659.0,90879.0,133.41,29.0,T,"Montgomery, AL"


# CT update

In [6]:
df_ct_total = df_census_2024[(df_census_2024['State' ] == '09')]

df_ct_old = df_census_2024[(df_census_2024['State' ] == '09') & (df_census_2024['County'].astype(int) <= 15)]
df_ct_new = df_census_2024[(df_census_2024['State' ] == '09') & (df_census_2024['County'].astype(int) > 15)]

df_census_2024_no_CT = df_census_2024[(df_census_2024['State' ] != '09')]



# Update Collection Year


In [7]:
df_census_2024_with_new_CT_county_codes = pd.concat([df_census_2024_no_CT, df_ct_new])
df_census_2024_with_new_CT_county_codes['Collection Year'] = 2024


# check counts

In [8]:
df_census_2024_no_CT.shape[0] + df_ct_new.shape[0] == df_census_2024_with_new_CT_county_codes.shape[0]

True

In [9]:
# if 'MSA/MD' is 99999 then 'MSA/MD Name' is "Not applicable"
df_census_2024_with_new_CT_county_codes.loc[(df_census_2024_with_new_CT_county_codes['MSA/MD'] == 99999), 'MSA/MD Name'] = np.NaN

In [10]:
df_census_2024_with_new_CT_county_codes[(df_census_2024_with_new_CT_county_codes['MSA/MD'] == 99999)].count()


Collection Year                   15403
MSA/MD                            15403
State                             15403
County                            15403
Census Tract                      15403
FFIEC Median Family Income        15403
Population                        15401
Minority Population %             15270
Number of Owner Occupied Units    15401
Number of 1 to 4 Family Units     15270
Tract MFI                         15380
Tract to MSA Income %             15403
Median Age                        15270
Small County                      15403
MSA/MD Name                           0
dtype: int64

In [11]:
df_census_2024_with_new_CT_county_codes[(df_census_2024_with_new_CT_county_codes['MSA/MD'] != 99999)].count()

Collection Year                   71896
MSA/MD                            71896
State                             71896
County                            71896
Census Tract                      71896
FFIEC Median Family Income        71896
Population                        71896
Minority Population %             71896
Number of Owner Occupied Units    71896
Number of 1 to 4 Family Units     71896
Tract MFI                         71896
Tract to MSA Income %             71896
Median Age                        71896
Small County                      71896
MSA/MD Name                       70884
dtype: int64

In [12]:
df_census_2024_with_new_CT_county_codes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87299 entries, 0 to 88181
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Collection Year                 87299 non-null  int64  
 1   MSA/MD                          87299 non-null  int64  
 2   State                           87299 non-null  object 
 3   County                          87299 non-null  object 
 4   Census Tract                    87299 non-null  int64  
 5   FFIEC Median Family Income      87299 non-null  int64  
 6   Population                      87297 non-null  float64
 7   Minority Population %           87166 non-null  float64
 8   Number of Owner Occupied Units  87297 non-null  float64
 9   Number of 1 to 4 Family Units   87166 non-null  float64
 10  Tract MFI                       87276 non-null  float64
 11  Tract to MSA Income %           87299 non-null  float64
 12  Median Age                      

# Write Output

In [13]:
df_census_2024_with_new_CT_county_codes.to_csv("../output_FHFA/" + "ffiec_census_{year}_with_new_CT_county_codes.{end}".format(year="2024", end="txt"), 
								   index=False, 
								   sep="|")

